In [1]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession

# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/12/25 13:36:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=701214;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=802986;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\gusta\AppData\Local\sagemaker\sagemaker\config.yaml


## Tabelas usadas pela query Montada com o Rene

In [2]:
cliente = spark.table("prd.dl_redeok_checkok.tb_cliente")
revenda = spark.table("prd.dl_redeok_checkok.tb_revenda")
receber = spark.table("prd.dl_redeok_checkok.vw_receber")

In [3]:
sd_cliente = cliente.select("matriz","codigo", "cgcmf","raz_social", "data_cad" , "bloqueio","mot_bloqueio","max_conexao", "sit_cliente","fis_jurid","vendedor","vlrfatmin", "vlr_dispacesso", "receita")
sd_cliente.show()



+------+------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+---------+--------+---------+--------------+-------+
|matriz|codigo|         cgcmf|          raz_social|            data_cad|            bloqueio|        mot_bloqueio|max_conexao|sit_cliente|fis_jurid|vendedor|vlrfatmin|vlr_dispacesso|receita|
+------+------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+---------+--------+---------+--------------+-------+
|297991|297991|07489900000193|LOCANORTE LOCACAO...|2014-06-20 00:00:...|2024-12-11 16:49:...|INADIMPL            |         -1|          N|        J|   LIBR5|        0|             0|     58|
|297991|349795|07489900000193|LOCACAO DE CACAMB...|2012-03-14 00:00:...|2024-12-11 16:49:...|INADIMPL            |         -1|          A|        J|   LIBR5|     NULL|          NULL|     58|
|297991|451146|07489900000193|LOCANORTE LOCAC

- Talvez esse max_conexão seja o previus status (possui o -5,-7 ,0 e -1)

In [4]:
revenda.show()
revenda.count()

+------+---------+-------+--------------------+----------+------------+---------------+-----------+--------------+--------+-------------+---------+--------+--------+--------------+
|CODIGO|DESCRICAO|REPASSE|          DTA_ULTATU|USR_ULTATU|COD_CONTRATO|DTA_ASSCONTRATO|TIP_REVENDA|COD_TABREPASSE|SG_GRUPO|VL_PERCENTUAL|SG_VISREP|SG_DEMCO|CD_CONTA|date_partition|
+------+---------+-------+--------------------+----------+------------+---------------+-----------+--------------+--------+-------------+---------+--------+--------+--------------+
|101549|    IVAN |      N|2010-08-25 11:23:...|  luzinete|        NULL|           NULL|          2|             1|    NULL|         NULL|        S|       N|    NULL|    2025-02-12|
| 95854|    JOAO |      N|2010-09-14 16:53:...|  luzinete|        NULL|           NULL|          2|             1|    NULL|         NULL|        S|       N|    NULL|    2025-02-12|
|259088|    OKABC|      S|2010-10-15 17:17:...|  luzinete|        NULL|           NULL|        

4172

In [5]:
receber.show(1000)
receber.count()

+-------+----------+-------+-------+-------------------+-------------------+-------------------+--------+-------+-------+-----+-------------------+----------+------------+--------------------+--------+----+-------------------+--------------------+--------+-----+----------+---------+----------+---------+-------+----+----+--------+------+-------+--------+----------+----------+----------+-----------+-----------+-----------+-----------+--------------------+--------+---------+-------------+-----------+-------------+---------------+--------+---------+------+--------+-------+----------+--------------------+---------+-------+---------+-----------+-----------------+--------------+-------+--------------+-------+-------+
| numero|    codigo|cliente|  valor|            emissao|         vencimento|          pagto_ate|desconto|receita|especie|banco|         data_pagto|valor_pago|cod_desconto|          observacao|situacao|tipo|       cancelamento|    mot_cancelamento|num_vias|baixa|quantidade|cadastr

5601405

In [6]:
df_filtrado = receber.filter(receber.cliente == "335400")

df_ordenado = df_filtrado.orderBy("emissao", ascending=True)


df_ordenado.show(100)
df_ordenado.count()


+-------+----------+-------+------+-------------------+-------------------+-------------------+--------+-------+-------+-----+-------------------+----------+------------+--------------------+--------+----+------------+----------------+--------+-----+----------+---------+----------+---------+-------+----+----+--------+------+-------+--------+----------+----------+----------+-----------+-----------+-----------+-----------+--------------------+--------+---------+-------------+-----------+-------------+---------------+--------+---------+-----+--------+-------+----------+---------+---------+-------+---------+-----------+-----------------+--------------+-------+--------------+-------+-------+
| numero|    codigo|cliente| valor|            emissao|         vencimento|          pagto_ate|desconto|receita|especie|banco|         data_pagto|valor_pago|cod_desconto|          observacao|situacao|tipo|cancelamento|mot_cancelamento|num_vias|baixa|quantidade|cadastral|vlr_renpac|vlr_dspac|repasse| mes

17

In [ ]:
query=  """WITH 
clientes AS 
(
    SELECT 
          COUNT(1)              AS contratos
        , cgcmf                 AS documento
        , MIN(data_cad)         AS data_inicio
        , MAX(bloqueio)         AS data_final
        , MAX_BY(mot_bloqueio, bloqueio) AS mot_bloqueio  
        , MAX_BY(max_conexao, bloqueio) AS max_conexao    
    FROM 
    (
        SELECT DISTINCT
            matriz, codigo, fis_jurid, cgcmf, raz_social, max_conexao, bloqueio, mot_bloqueio,
            sit_cliente, data_cad, vendedor, vlrfatmin, vlr_dispacesso, receita
        FROM 
            prd.dl_redeok_checkok.tb_cliente cli
        WHERE 
            cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14
            AND cli.cgcmf RLIKE '^[0-9]+$'
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE cli.codigo = rev.codigo
            )
    ) 
    GROUP BY cgcmf
),
titulo_cobranca AS 
(
    SELECT 
          documento         AS documento
        , cliente          AS cliente  
        , COUNT(meses)      AS boletos_gerados
    FROM 
    (
        SELECT DISTINCT
            cli.cgcmf                           AS documento
            , rec.cliente                       AS cliente  
            , DATE_TRUNC('month', rec.emissao)  AS meses
        FROM 
            prd.dl_redeok_checkok.vw_receber rec
        INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
            ON cli.codigo = rec.cliente
        WHERE 
            rec.situacao IN ('A', 'B')
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE rev.codigo = rec.cliente
            )
            AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14    
            AND cli.cgcmf RLIKE '^[0-9]+$'
    ) 
    GROUP BY 
        documento, cliente  
),
ultimo_boleto AS 
(
    SELECT 
        cli.cgcmf AS documento,
        MAX(rec.emissao) AS ultima_emissao
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
),
soma_valor_pago AS
(
    SELECT 
        cli.cgcmf AS documento,
        SUM(rec.valor_pago) AS total_valor_pago
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
),
documentos_filtrados AS (
    SELECT 
          cli.contratos
        , cli.documento
        , tit.cliente  
        , cli.data_inicio
        , cli.data_final
        , cli.mot_bloqueio   
        , cli.max_conexao    
        , CASE 
              WHEN cli.data_final IS NULL THEN NULL
              ELSE ult.ultima_emissao
          END AS data_ultimo_boleto
        , tit.boletos_gerados
        , ROUND(MONTHS_BETWEEN(COALESCE(cli.data_final, CURRENT_DATE()), cli.data_inicio)) AS meses_contrato
        , ROUND(COALESCE(soma.total_valor_pago, 0), 1) AS total_valor_pago
        , ROW_NUMBER() OVER (PARTITION BY cli.documento ORDER BY tit.boletos_gerados DESC) AS row_number
    FROM 
        clientes cli
    INNER JOIN titulo_cobranca tit 
        ON cli.documento = tit.documento
    LEFT JOIN ultimo_boleto ult
        ON cli.documento = ult.documento
    LEFT JOIN soma_valor_pago soma
        ON cli.documento = soma.documento
)
SELECT * 
FROM documentos_filtrados
WHERE row_number = 1;


"""


# Executando a consulta e obtendo os dados em um DataFrame Spark
tabela_nova_sdf= spark.sql(query)

from pyspark.sql.functions import round

tabela_nova_sdf = tabela_nova_sdf.withColumn("anos_boletos_gerados", round(col("boletos_gerados") / 12, 2))
tabela_nova_sdf = tabela_nova_sdf.withColumn("anos_contrato", round(col("meses_contrato") / 12, 2))

tabela_nova_sdf = tabela_nova_sdf.withColumn("churn",when(col("data_final").isNotNull(), 1).otherwise(0))


In [8]:
tabela_nova_sdf.show()

+---------+--------------+-------+--------------------+--------------------+--------------------+-----------+--------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|contratos|     documento|cliente|         data_inicio|          data_final|        mot_bloqueio|max_conexao|  data_ultimo_boleto|boletos_gerados|meses_contrato|total_valor_pago|anos_boletos_gerados|anos_contrato|churn|
+---------+--------------+-------+--------------------+--------------------+--------------------+-----------+--------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|        1|11342330000181| 449644|2021-02-10 11:39:...|2022-06-21 00:00:...|NAO UTIL. SERV      |         -1| 2022-06-01 00:00:00|             15|          16.0|           805.4|                1.25|         1.33|    1|
|        1|01834619000164| 335400|2013-12-05 11:44:...|2015-05-13 16:44:...|MOTIVO NAO INFORMADO|         -1| 2015-05-01

In [10]:
tabela_nova_sdf.count()

136731

In [9]:
from pyspark.sql.functions import sum, col

# Contar o número total de clientes na tabela
contagem_total = tabela_nova_sdf.select("documento").distinct().count()

# Somar o número de contratos por cliente
contratos_por_cliente = (
    tabela_nova_sdf.groupBy("documento")
    .agg(sum("contratos").alias("quantidade_contratos"))  # Alterado de count() para sum()
)

# Contar quantos clientes têm apenas um contrato e quantos têm mais de um contrato
clientes_um_contrato = contratos_por_cliente.filter(col("quantidade_contratos") == 1).count()
clientes_mais_um_contrato = contratos_por_cliente.filter(col("quantidade_contratos") > 1).count()

# Calcular a receita total (soma de valor_pago para todos os clientes)
receita_total = tabela_nova_sdf.select(sum("total_valor_pago")).collect()[0][0]

# Calcular a receita apenas para clientes com um contrato
receita_um_contrato = (
    tabela_nova_sdf.alias("t")
    .join(contratos_por_cliente.alias("c"), col("t.documento") == col("c.documento"))
    .filter(col("c.quantidade_contratos") == 1)
    .select(sum("t.total_valor_pago"))
    .collect()[0][0]
)

# Calcular a receita para clientes com mais de um contrato
receita_mais_um_contrato = (
    tabela_nova_sdf.alias("t")
    .join(contratos_por_cliente.alias("c"), col("t.documento") == col("c.documento"))
    .filter(col("c.quantidade_contratos") > 1)
    .select(sum("t.total_valor_pago"))
    .collect()[0][0]
)

from pyspark.sql.functions import sum, col, when

# Somar o número de contratos por cliente
contratos_por_cliente = (
    tabela_nova_sdf.groupBy("documento")
    .agg(sum("contratos").alias("quantidade_contratos"))
)

# Calcular a soma dos contratos para clientes com apenas um contrato e mais de um contrato
quantidade_contratos_por_grupo = contratos_por_cliente.groupBy(
    when(col("quantidade_contratos") == 1, "um_contrato")
    .otherwise("mais_de_um_contrato")
    .alias("tipo_cliente")
).agg(
    sum("quantidade_contratos").alias("quantidade_total_contratos")
)

# Coletar os resultados
resultado = quantidade_contratos_por_grupo.collect()

contratos_um_contrato = next((row["quantidade_total_contratos"] for row in resultado if row["tipo_cliente"] == "um_contrato"), 0)
contratos_mais_um_contrato = next((row["quantidade_total_contratos"] for row in resultado if row["tipo_cliente"] == "mais_de_um_contrato"), 0)

# Exibir os resultados


# Exibir o resultado final
print(f"Quantidade total de clientes: {contagem_total}")
print(f"Clientes com apenas um contrato: {clientes_um_contrato}")
print(f"Clientes com mais de um contrato: {clientes_mais_um_contrato}")
print(f"Receita total: {receita_total:.1f}")
print(f"Receita de clientes com apenas um contrato: {receita_um_contrato:.1f}")
print(f"Receita de clientes com mais de um contrato: {receita_mais_um_contrato:.1f}")
print(f"Total de contratos de clientes com apenas um contrato: {contratos_um_contrato}")
print(f"Total de contratos de clientes com mais de um contrato: {contratos_mais_um_contrato}")


Quantidade total de clientes: 134316
Clientes com apenas um contrato: 130720
Clientes com mais de um contrato: 3596
Receita total: 712879058.1
Receita de clientes com apenas um contrato: 635430508.6
Receita de clientes com mais de um contrato: 77448549.5
Total de contratos de clientes com apenas um contrato: 130720
Total de contratos de clientes com mais de um contrato: 12771


635 milhoes e 41 milhões

In [9]:
from pyspark.sql import functions as F

# Contar quantidade de clientes que cancelaram e não cancelaram
churn_counts = tabela_nova_sdf.groupBy("churn").count()

# Exibir o resultado
churn_counts.show()


+-----+------+
|churn| count|
+-----+------+
|    1|120953|
|    0| 13363|
+-----+------+



In [15]:
# Filtrando os registros onde 'meses_contrato_anos' é negativo
clientes_com_meses_negativos = tabela_nova_sdf.filter(col("meses_contrato") <= 0)

# Contando a quantidade de clientes
quantidade_clientes = clientes_com_meses_negativos.count()

# Exibindo o resultado
print(f"Quantidade de clientes com 'meses_contrato' negativo: {quantidade_clientes}")

# Exibindo os clientes com meses negativos
clientes_com_meses_negativos.show(2000,truncate=False)

Quantidade de clientes com 'meses_contrato' negativo: 1642
+---------+--------------+-----------------------+-----------------------+--------------------+-----------+---------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|contratos|documento     |data_inicio            |data_final             |mot_bloqueio        |max_conexao|data_ultimo_boleto   |boletos_gerados|meses_contrato|total_valor_pago|anos_boletos_gerados|anos_contrato|churn|
+---------+--------------+-----------------------+-----------------------+--------------------+-----------+---------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|1        |07300030000162|2008-05-20 00:00:00.000|0012-05-21 00:00:00.000|CONTRATO CANCELADO  |-1         |2012-05-01 00:00:00  |48             |-23952.0      |5278.0          |4.0                 |-1996.0      |1    |
|1        |82255514000152|2007-10-25 00:00:00.000|0012-05-25 00:0

In [35]:
sdf_ordenado = tabela_nova_sdf.orderBy(col("contratos").desc())
sdf_ordenado.show()

+---------+--------------+-------+--------------------+--------------------+--------------------+-----------+--------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|contratos|     documento|cliente|         data_inicio|          data_final|        mot_bloqueio|max_conexao|  data_ultimo_boleto|boletos_gerados|meses_contrato|total_valor_pago|anos_boletos_gerados|anos_contrato|churn|
+---------+--------------+-------+--------------------+--------------------+--------------------+-----------+--------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|       84|08838057000176| 345200|2014-04-23 00:00:...|2020-02-19 17:02:...|SOLICITACAO REVENDA |         -1|2024-11-01 00:00:...|             84|          70.0|         46094.0|                 7.0|         5.83|    1|
|       26|12531484000184| 217037|2010-10-27 00:00:...|2023-02-03 00:00:...|LOGON CORROMPIDO    |         -1|2016-04-01 

### Apenas Clientes com 1 contrato e eliminamos datas iregulares ou clientes que tem uma permanencia de 0 dias

In [16]:
from pyspark.sql.functions import col

from pyspark.sql.functions import col

# Filtrando apenas os documentos que têm contratos = 1 e removendo os contratos com meses negativos
tabela_filtrada_sdf = tabela_nova_sdf.filter((col("contratos") == 1) & (col("meses_contrato") > 0))

# Exibindo o resultado
tabela_filtrada_sdf.show(truncate=False)  # Use truncate=False para ver os dados completos



+---------+--------------+-----------------------+-----------------------+--------------------+-----------+---------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|contratos|documento     |data_inicio            |data_final             |mot_bloqueio        |max_conexao|data_ultimo_boleto   |boletos_gerados|meses_contrato|total_valor_pago|anos_boletos_gerados|anos_contrato|churn|
+---------+--------------+-----------------------+-----------------------+--------------------+-----------+---------------------+---------------+--------------+----------------+--------------------+-------------+-----+
|1        |67902890000106|2003-08-21 09:36:53.0  |NULL                   |NULL                |NULL       |NULL                 |231            |258.0         |9541.7          |19.25               |21.5         |0    |
|1        |07047708000147|2014-01-13 00:00:00.000|2015-05-05 00:00:00.000|FINANCEIRO/SERASA   |-3         |2015-06-06 00:00:

In [ ]:
from pyspark.sql.functions import sum, col

# Contar o número total de clientes na tabela
contagem_total = tabela_filtrada_sdf.select("documento").distinct().count()

# Somar o número de contratos por cliente
contratos_por_cliente = (
    tabela_filtrada_sdf.groupBy("documento")
    .agg(sum("contratos").alias("quantidade_contratos"))  # Alterado de count() para sum()
)

# Contar quantos clientes têm apenas um contrato e quantos têm mais de um contrato
clientes_um_contrato = contratos_por_cliente.filter(col("quantidade_contratos") == 1).count()
clientes_mais_um_contrato = contratos_por_cliente.filter(col("quantidade_contratos") > 1).count()

# Calcular a receita total (soma de valor_pago para todos os clientes)
receita_total = tabela_filtrada_sdf.select(sum("total_valor_pago")).collect()[0][0]

# Calcular a receita apenas para clientes com um contrato
receita_um_contrato = (
    tabela_filtrada_sdf.alias("t")
    .join(contratos_por_cliente.alias("c"), col("t.documento") == col("c.documento"))
    .filter(col("c.quantidade_contratos") == 1)
    .select(sum("t.total_valor_pago"))
    .collect()[0][0]
)

# Calcular a receita para clientes com mais de um contrato
receita_mais_um_contrato = (
    tabela_filtrada_sdf.alias("t")
    .join(contratos_por_cliente.alias("c"), col("t.documento") == col("c.documento"))
    .filter(col("c.quantidade_contratos") > 1)
    .select(sum("t.total_valor_pago"))
    .collect()[0][0]
)

from pyspark.sql.functions import sum, col, when

# Somar o número de contratos por cliente
contratos_por_cliente = (
    tabela_filtrada_sdf.groupBy("documento")
    .agg(sum("contratos").alias("quantidade_contratos"))
)

# Calcular a soma dos contratos para clientes com apenas um contrato e mais de um contrato
quantidade_contratos_por_grupo = contratos_por_cliente.groupBy(
    when(col("quantidade_contratos") == 1, "um_contrato")
    .otherwise("mais_de_um_contrato")
    .alias("tipo_cliente")
).agg(
    sum("quantidade_contratos").alias("quantidade_total_contratos")
)

# Coletar os resultados
resultado = quantidade_contratos_por_grupo.collect()

contratos_um_contrato = next((row["quantidade_total_contratos"] for row in resultado if row["tipo_cliente"] == "um_contrato"), 0)
contratos_mais_um_contrato = next((row["quantidade_total_contratos"] for row in resultado if row["tipo_cliente"] == "mais_de_um_contrato"), 0)

# Exibir os resultados


# Exibir o resultado final
print(f"Quantidade total de clientes: {contagem_total}")
print(f"Clientes com apenas um contrato: {clientes_um_contrato}")
print(f"Clientes com mais de um contrato: {clientes_mais_um_contrato}")
print(f"Receita total: {receita_total:.1f}")
print(f"Receita de clientes com apenas um contrato: {receita_um_contrato:.1f}")
print(f"Total de contratos de clientes com apenas um contrato: {contratos_um_contrato}")
#print(f"Receita de clientes com mais de um contrato: {receita_mais_um_contrato:.1f}")
#print(f"Total de contratos de clientes com mais de um contrato: {contratos_mais_um_contrato}")



Quantidade total de clientes: 129098
Clientes com apenas um contrato: 129098
Clientes com mais de um contrato: 0
Receita total: 633413997.5
Receita de clientes com apenas um contrato: 633413997.5
Total de contratos de clientes com apenas um contrato: 129098


In [18]:
# Contar quantidade de clientes que cancelaram e não cancelaram
churn_counts = tabela_filtrada_sdf.groupBy("churn").count()

# Exibir o resultado
churn_counts.show()

+-----+------+
|churn| count|
+-----+------+
|    1|115771|
|    0| 13327|
+-----+------+



In [19]:
import plotly.graph_objects as go
import pandas as pd

# Converter para Pandas para visualização
df_permanencia = tabela_filtrada_sdf.select("anos_contrato", "data_final").toPandas()

# Garantir que há valores para definir o bin máximo
max_anos = df_permanencia["anos_contrato"].max()
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, max_anos + 1]  # Adiciona um espaço extra para o último bin
labels = [
    "0-1 ano",
    "1-2 anos",
    "2-3 anos",
    "3-4 anos",
    "4-5 anos",
    "5-6 anos",
    "6-7 anos",
    "7-8 anos",
    "8-9 anos",
    "9-10 anos",
    "10+ anos"
]

# Criar coluna categórica com os intervalos corrigidos
df_permanencia["faixa_permanencia"] = pd.cut(df_permanencia["anos_contrato"], bins=bins, labels=labels, right=False)

# Classificar contratos como Ativo (nulo) ou Cancelado (não nulo)
df_permanencia["status_contrato"] = df_permanencia["data_final"].apply(lambda x: "Ativo" if pd.isna(x) else "Cancelado")

# Contar a quantidade de contratos ativos e cancelados por faixa de permanência
df_faixas_status = df_permanencia.groupby(["faixa_permanencia", "status_contrato"]).size().unstack(fill_value=0)

# Criar gráfico de barras agrupadas (um do lado do outro)
fig = go.Figure()

# Adicionar barras para Ativo e Cancelado
for status in ["Ativo", "Cancelado"]:
    fig.add_trace(go.Bar(
        x=df_faixas_status.index.astype(str),
        y=df_faixas_status[status],
        name=status
    ))

# Configurar layout do gráfico
fig.update_layout(
    title="Distribuição do Tempo de Permanência dos Clientes (Diferença Datas)",
    xaxis_title="Faixa de Tempo de Permanência (anos)",
    yaxis_title="Quantidade de Clientes (escala logarítmica)",
    barmode="group",  # Barras lado a lado
    yaxis=dict(type="log")  # Aplicar escala logarítmica
)

# Exibir gráfico
fig.show()


# Calcular a porcentagem de cada status dentro de cada faixa de permanência
df_faixas_status_percent = df_faixas_status.div(df_faixas_status.sum(axis=1), axis=0) * 100

df_faixas_status_percent = df_faixas_status_percent.round(2).astype(str) + '%'
# Exibir a tabela de faixas de permanência com status
display(df_faixas_status)
display(df_faixas_status_percent)


C:\Users\gusta\AppData\Local\Temp\ipykernel_9160\338459736.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_faixas_status = df_permanencia.groupby(["faixa_permanencia", "status_contrato"]).size().unstack(fill_value=0)


status_contrato,Ativo,Cancelado
faixa_permanencia,,
0-1 ano,1110,25466
1-2 anos,1033,24618
2-3 anos,793,14666
3-4 anos,878,10614
4-5 anos,738,8772
5-6 anos,1034,6900
6-7 anos,624,5386
7-8 anos,562,4087
8-9 anos,839,3517


status_contrato,Ativo,Cancelado
faixa_permanencia,,
0-1 ano,4.18%,95.82%
1-2 anos,4.03%,95.97%
2-3 anos,5.13%,94.87%
3-4 anos,7.64%,92.36%
4-5 anos,7.76%,92.24%
5-6 anos,13.03%,86.97%
6-7 anos,10.38%,89.62%
7-8 anos,12.09%,87.91%
8-9 anos,19.26%,80.74%


In [20]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = tabela_filtrada_sdf.select("data_inicio", "anos_contrato").toPandas()

# Converter start_date para datetime
df_permanencia["data_inicio"] = pd.to_datetime(df_permanencia["data_inicio"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["data_inicio"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular estatísticas por ano
estatisticas_por_ano = df_permanencia.groupby("ano_inicio")["anos_contrato"].agg(["median", "mean"]).reset_index()

# Calcular o tempo máximo possível para cada ano (tempo entre ano de início e hoje)
estatisticas_por_ano["tempo_maximo_possivel"] = ano_atual - estatisticas_por_ano["ano_inicio"]

# Calcular a diferença entre o tempo máximo e a mediana
estatisticas_por_ano["diferenca"] = estatisticas_por_ano["tempo_maximo_possivel"] - estatisticas_por_ano["median"]

# Identificar o primeiro ano onde a diferença for menor que um limite (exemplo: 2 anos)
#ano_limite_vies = estatisticas_por_ano.loc[estatisticas_por_ano["diferenca"] < 2, "ano_inicio"].min()

# Criar gráfico interativo
fig = go.Figure()

# Linha da mediana do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["median"],
    mode="lines+markers",
    name="Mediana do Tempo de Permanência",
    line=dict(color="blue")
))

# Linha da média do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["mean"],
    mode="lines+markers",
    name="Média do Tempo de Permanência",
    line=dict(color="green")
))

# Linha do tempo máximo possível
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["tempo_maximo_possivel"],
    mode="lines",
    name="Tempo Máximo Possível",
    line=dict(color="red", dash="dash")
))



# Layout do gráfico
fig.update_layout(
    title="Detecção do Viés no Tempo de Permanência",
    xaxis_title="Ano de Início do Cliente",
    yaxis_title="Tempo de Permanência (anos)",
    showlegend=True
)

# Exibir gráfico
fig.show()




In [23]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = tabela_filtrada_sdf.select("data_inicio", "anos_contrato").toPandas()

# Converter data_inicio para datetime
df_permanencia["data_inicio"] = pd.to_datetime(df_permanencia["data_inicio"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["data_inicio"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular o tempo máximo possível para cada ano
df_permanencia["tempo_maximo_possivel"] = ano_atual - df_permanencia["ano_inicio"]

# Criar Boxplot simplificado com menos categorias no eixo X
fig = px.box(
    df_permanencia,
    x="ano_inicio",  # Agora usamos apenas o ano, reduzindo o número de categorias no eixo X
    y="anos_contrato",
    points="all",
    title="Distribuição do Tempo de Permanência por Ano de Início do Contrato",
    labels={"ano_inicio": "Ano de Início do Contrato", "anos_contrato": "Tempo de Permanência (anos)"},
)

# Adicionar linha guia do tempo máximo possível
fig.add_trace(
    go.Scatter(
        x=df_permanencia["ano_inicio"].unique(),
        y=(ano_atual - df_permanencia["ano_inicio"].unique()),  
        mode="lines",
        name="Tempo Máximo Possível",
        line=dict(color="red", dash="dash")
    )
)

# Exibir gráfico
fig.show()


In [21]:
from pyspark.sql import functions as F

# Criar a coluna do ano de início do contrato
sdf_churn_anual = tabela_filtrada_sdf.withColumn("ano_inicio", F.year("data_inicio")) \
    .groupBy("ano_inicio") \
    .agg(
        F.count("*").alias("total_clientes"),
        F.sum(F.when(F.col("data_final").isNotNull(), 1).otherwise(0)).alias("contratos_cancelados")
    ) \
    .withColumn("taxa_churn", (F.col("contratos_cancelados") / F.col("total_clientes")) * 100) \
    .orderBy("ano_inicio")

# Converter para Pandas para plotar
df_churn_anual = sdf_churn_anual.toPandas()

# Criar gráfico interativo com Plotly
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_churn_anual["ano_inicio"],
    y=df_churn_anual["taxa_churn"],
    marker_color="red",
    name="Taxa de Churn (%)"
))

# Ajustes de layout
fig.update_layout(
    title="Taxa de Churn Anual",
    xaxis_title="Ano de Início do Contrato",
    yaxis_title="Taxa de Churn (%)",
    template="plotly_white",
    xaxis=dict(tickmode="linear")
)

# Exibir o gráfico
fig.show()


In [26]:
import plotly.graph_objects as go

def plot_cancelamentos_por_ano_plotly(spark_df):
    """
    Plota um gráfico de barras interativo usando Plotly, mostrando a quantidade de clientes cancelados por ano.
    
    Parâmetros:
    spark_df (pyspark.sql.DataFrame): DataFrame do Spark contendo a coluna 'data_final'.
    """
    from pyspark.sql.functions import col, year

    # Filtrando clientes cancelados (data_final NÃO é nula)
    clientes_cancelados = spark_df.filter(col("data_final").isNotNull())

    # Extraindo o ano da data_final
    clientes_cancelados = clientes_cancelados.withColumn("ano_cancelamento", year(col("data_final")))

    # Contando a quantidade de clientes cancelados por ano
    contagem_cancelados_por_ano = clientes_cancelados.groupBy("ano_cancelamento").count()

    # Convertendo para Pandas para plotar com Plotly
    df_cancelados = contagem_cancelados_por_ano.toPandas().sort_values("ano_cancelamento")

    # Criando o gráfico interativo com Plotly
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df_cancelados["ano_cancelamento"],
        y=df_cancelados["count"],
        marker_color="red",
        text=df_cancelados["count"],
        textposition='auto'
    ))

    # Adicionando títulos e rótulos
    fig.update_layout(
        title="Quantidade de Clientes Cancelados por Ano",
        xaxis_title="Ano de Cancelamento",
        yaxis_title="Quantidade de Cancelamentos",
        xaxis=dict(tickmode="linear"),
        template="plotly_white"
    )

    # Exibindo o gráfico
    fig.show()

# Para rodar a função, basta passar o DataFrame do Spark:
plot_cancelamentos_por_ano_plotly(tabela_filtrada_sdf)


## Tabelas usadas para a querry de Churn

In [19]:
churn_sdf_transicao = spark.table("dev.dw_redeok_model_aux.tb_s_status_transicao")
churn_sdf_ass = spark.table("dev.dw_redeok_model.tb_d_external_entity_association")
churn_sdf_contrato = spark.table("dev.dw_redeok_model.tb_d_contract")
churn_sdf_cliente = spark.table("dev.dw_redeok_model.tb_d_customer")
churn_sdf = spark.table("dev.dw_redeok_model.tb_f_churn")

In [ ]:
query_global=  """


with

transicao_global as
(
      select
              cnt.uuid                                                                        as uuid_contract
            , str.matriz                                                                      as matriz
            , str.data_transicao                                                              as data_transicao
            , str.previous_status                                                             as previous_status
            , str.next_status                                                                 as next_status
            , str.motivo_cancelamento                                                         as motivo_cancelamento
      from
            dev.dw_redeok_model_aux.tb_s_status_transicao str
            --
            left join dev.dw_redeok_model.tb_d_external_entity_association ext 
                  on str.matriz = ext.id_external_entity
                  and name_internal_entity = 'tb_d_contract'
                  and ext.name_external_application = 'signet'
            --
            left join dev.dw_redeok_model.tb_d_contract cnt
                  on ext.uuid_internal_entity = cnt.uuid
),
bloqueio as (
  select
           uuid_contract
        ,  matriz
        , data_transicao
        , previous_status
        , next_status
        , motivo_cancelamento
        , row_number() over (partition by matriz order by data_transicao desc)                as row_num
  from
        transicao_global
        qualify row_number() over (partition by matriz order by data_transicao desc) = 1
),

cancelamento_noventa_dias_bloqueio as 
(
  select 
          uuid_contract                                                                       as uuid_contract
        , matriz                                                                              as matriz
        , data_transicao                                                                      as data_transicao
        , dateadd(DAY, 90, data_transicao)                                                    as data_transicao_90d
        , previous_status                                                                     as previous_status
        , next_status                                                                         as next_status
        , motivo_cancelamento                                                                 as motivo_cancelamento
  from
        bloqueio
  where
        next_status in (-7, -5, 0)
),
cancelamento_normal_bloqueio_90d as
(
  select
          uuid_contract                                                                       as uuid_contract
        , matriz                                                                              as matriz
        , data_transicao                                                                      as churn_date
        , previous_status                                                                     as previous_status
        , next_status                                                                         as next_status
        , motivo_cancelamento                                                                 as motivo_cancelamento
        , 'CANCELAMENTO NORMAL'                                                               as tipo_saida
  from
      transicao_global
  where
        next_status = -1

union

select
        uuid_contract                                                                          as uuid_contract
      , matriz                                                                                 as matriz
      , data_transicao_90d                                                                     as churn_date
      , previous_status                                                                        as previous_status
      , next_status                                                                            as next_status
      , motivo_cancelamento                                                                    as motivo_cancelamento
      , case
            --when date_add(data_transicao_90d, 1) > current_date()         then 'PROJEÇÃO BLOQUEIO'
            when next_status = 0                                          then '90 DIAS BLOQUEADO'
            when next_status = -5                                         then '90 DIAS PRÉ CANCELADO'
            when next_status = -7                                         then '90 DIAS PRÉ CANCELADO JURÍDICO'
            else null
      end                                                                                      as tipo_saida
from
      cancelamento_noventa_dias_bloqueio
where
      date(data_transicao_90d) < current_date()

)
select
        cast(uuid() as string)                                                                  as uuid
      , cnt.uuid                                                                                as uuid_contract
      , cnt.uuid_customer                                                                       as uuid_customer
      , cast(can.churn_date as timestamp)                                                       as churn_date
      , trim(can.motivo_cancelamento)                                                           as motivation
      , can.tipo_saida                                                                          as churn_type
      , 'signet'                                                                                as data_source_insert
      , now()                                                                                   as created_at
      , ''                                                                                      as data_source_update
      , now()                                                                                   as updated_at
from
      cancelamento_normal_bloqueio_90d can
      --
      inner join dev.dw_redeok_model.tb_d_contract cnt
             on can.uuid_contract = cnt.uuid
      --
      inner join dev.dw_redeok_model.tb_d_customer as cus
            on cnt.uuid_customer = cus.uuid
where
      date(can.churn_date) >= date('2024-04-16') -- ultima data utilizada com base exportada do bigquery 
      and upper(trim(can.motivo_cancelamento)) not in ( 
                                                            'TROCA DE CNPJ',
                                                            'TROCA CNPJ',
                                                            'TROCA DE RECEITA',
                                                            'TROCA DE CARTEIRA',
                                                            'TROCA RECEITA',
                                                            'TROCA DE MODALIDADE')
     
      order by can.uuid_contract desc


      """


# Executando a consulta e obtendo os dados em um DataFrame Spark
query_global= spark.sql(query_global)
query_global = query_global.orderBy(col("churn_date"))
query_global.show()

contagem_total =query_global.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_global.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")

In [20]:
churn_sdf_transicao.show()

+--------+------+--------------------+---------------+-----------+--------------------+
| num_seq|matriz|      data_transicao|previous_status|next_status| motivo_cancelamento|
+--------+------+--------------------+---------------+-----------+--------------------+
|23362930|485659|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362919|454006|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362927|483225|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362922|466654|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362929|484557|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362920|454111|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362924|470126|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362925|470336|2024-10-02 02:00:...|              1|          0| BLOQ. AUTO. 15 DIAS|
|23362926|471394|2024-10-02 02:0

In [24]:
from pyspark.sql.functions import col

# Obter os valores distintos de next_status para cada motivo_cancelamento
relacao_status_motivo = (
    churn_sdf_transicao
    .select("next_status", "motivo_cancelamento")
    .distinct()
    .orderBy("motivo_cancelamento", "next_status")  # Ordenação para melhor visualização
)


relacao_status_motivo.show(100000)

+-----------+--------------------+
|next_status| motivo_cancelamento|
+-----------+--------------------+
|         -1|                NULL|
|         -2|                NULL|
|         -3|                NULL|
|         -5|                NULL|
|         -6|                NULL|
|         -7|                NULL|
|          0|                NULL|
|          1|                NULL|
|         -1|                    |
|         -1|      'PRE CANCELADO|
|         -5|      'PRE CANCELADO|
|          0|             ,02E+13|
|         -1|                   .|
|         -7|                   .|
|          0|                   .|
|         -1| .CONTRATO CANCELADO|
|         -5|       .RE CANCELADO|
|          0|        000000000000|
|         -1|006 - COMP OK SEGURO|
|         -1|007 - CONC BOA VISTA|
|         -1|   008 - CONC SERADA|
|         -5|   008 - CONC SERADA|
|         -1|   008 - CONC SERASA|
|         -5|   008 - CONC SERASA|
|         -1|  008 - CONC SERASAS|
|         -5|  008 -